In [1]:
versioninfo()

Julia Version 1.4.1
Commit 381693d3df* (2020-04-14 17:20 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, haswell)
Environment:
  JULIA_PATH = /srv/julia
  JULIA_DEPOT_PATH = /srv/julia/pkg
  JULIA_PROJECT = /home/jovyan
  JULIA_VERSION = 1.4.1


## Naive 実装

In [2]:
function tarai_naive(x, y, z)
    if x ≤ y
        y
    else
        tarai_naive(tarai_naive(x - 1, y, z), tarai_naive(y - 1, z, x), tarai_naive(z - 1, x, y))
    end
end

tarai_naive (generic function with 1 method)

In [3]:
tarai_naive(4, 2, 0)

4

In [4]:
@time tarai_naive(12, 6, 0)

  0.038978 seconds


12

In [5]:
@time tarai_naive(14, 7, 0)

  1.953000 seconds


14

In [6]:
@time tarai_naive(16, 8, 0)

115.857705 seconds


16

In [ ]:
# tarai_naive(100, 50, 0)

## メモ化（手実装）

In [7]:
const TARAI_CACHE = Dict{NTuple{3, Int}, Int}()

Dict{Tuple{Int64,Int64,Int64},Int64} with 0 entries

In [8]:
function tarai_cached(x, y, z)
    get!(TARAI_CACHE, (x, y, z)) do
        if x ≤ y
            y
        else
            tarai_cached(tarai_cached(x - 1, y, z), tarai_cached(y - 1, z, x), tarai_cached(z - 1, x, y))
        end
    end
end

tarai_cached (generic function with 1 method)

In [9]:
tarai_cached(4, 2, 0)

4

In [10]:
@time tarai_cached(12, 6, 0)

  0.000055 seconds (7 allocations: 41.953 KiB)


12

In [11]:
@time tarai_cached(14, 7, 0)

  0.000046 seconds


14

In [12]:
@time tarai_cached(16, 8, 0)

  0.000025 seconds


16

In [13]:
@time tarai_cached(100, 50, 0)

  0.013841 seconds (17 allocations: 2.708 MiB)


100

## Naive 実装 と メモ化（手実装） の AST 比較

In [14]:
ex_tarai_naive = :(function tarai_naive(x, y, z)
    if x ≤ y
        y
    else
        tarai_naive(tarai_naive(x - 1, y, z), tarai_naive(y - 1, z, x), tarai_naive(z - 1, x, y))
    end
end)

:(function tarai_naive(x, y, z)
      #= In[14]:2 =#
      if x ≤ y
          #= In[14]:3 =#
          y
      else
          #= In[14]:5 =#
          tarai_naive(tarai_naive(x - 1, y, z), tarai_naive(y - 1, z, x), tarai_naive(z - 1, x, y))
      end
  end)

In [15]:
ex_tarai_cached = :(function tarai_cached(x, y, z)
    get!(TARAI_CACHE, (x, y, z)) do
        if x ≤ y
            y
        else
            tarai_cached(tarai_cached(x - 1, y, z), tarai_cached(y - 1, z, x), tarai_cached(z - 1, x, y))
        end
    end
end)

:(function tarai_cached(x, y, z)
      #= In[15]:2 =#
      get!(TARAI_CACHE, (x, y, z)) do 
          #= In[15]:3 =#
          if x ≤ y
              #= In[15]:4 =#
              y
          else
              #= In[15]:6 =#
              tarai_cached(tarai_cached(x - 1, y, z), tarai_cached(y - 1, z, x), tarai_cached(z - 1, x, y))
          end
      end
  end)

In [16]:
dump(ex_tarai_naive)

Expr
  head: Symbol function
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((4,))
        1: Symbol tarai_naive
        2: Symbol x
        3: Symbol y
        4: Symbol z
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 2
          file: Symbol In[14]
        2: Expr
          head: Symbol if
          args: Array{Any}((3,))
            1: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ≤
                2: Symbol x
                3: Symbol y
            2: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Symbol y
            3: Expr
              head: Symbol block
              args: Array{Any}((2,))
                1: LineNumberNode
                2: Expr


In [17]:
# dump(ex_tarai_cached)
dump(ex_tarai_cached; maxdepth=14)

Expr
  head: Symbol function
  args: Array{Any}((2,))
    1: Expr
      head: Symbol call
      args: Array{Any}((4,))
        1: Symbol tarai_cached
        2: Symbol x
        3: Symbol y
        4: Symbol z
    2: Expr
      head: Symbol block
      args: Array{Any}((2,))
        1: LineNumberNode
          line: Int64 2
          file: Symbol In[15]
        2: Expr
          head: Symbol do
          args: Array{Any}((2,))
            1: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol get!
                2: Symbol TARAI_CACHE
                3: Expr
                  head: Symbol tuple
                  args: Array{Any}((3,))
                    1: Symbol x
                    2: Symbol y
                    3: Symbol z
            2: Expr
              head: Symbol ->
              args: Array{Any}((2,))
                1: Expr
                  head: Symbol tuple
                  args: Array{Any}((0,))
                2: Expr


In [18]:
(ex_tarai_naive.args[1], ex_tarai_cached.args[1])

(:(tarai_naive(x, y, z)), :(tarai_cached(x, y, z)))

In [19]:
ex_tarai_naive.args[2]

quote
    #= In[14]:2 =#
    if x ≤ y
        #= In[14]:3 =#
        y
    else
        #= In[14]:5 =#
        tarai_naive(tarai_naive(x - 1, y, z), tarai_naive(y - 1, z, x), tarai_naive(z - 1, x, y))
    end
end

In [20]:
ex_tarai_cached.args[2].args[2].args[2].args[2]

quote
    #= In[15]:3 =#
    if x ≤ y
        #= In[15]:4 =#
        y
    else
        #= In[15]:6 =#
        tarai_cached(tarai_cached(x - 1, y, z), tarai_cached(y - 1, z, x), tarai_cached(z - 1, x, y))
    end
end

In [21]:
ex_tarai_cached.args[2]

quote
    #= In[15]:2 =#
    get!(TARAI_CACHE, (x, y, z)) do 
        #= In[15]:3 =#
        if x ≤ y
            #= In[15]:4 =#
            y
        else
            #= In[15]:6 =#
            tarai_cached(tarai_cached(x - 1, y, z), tarai_cached(y - 1, z, x), tarai_cached(z - 1, x, y))
        end
    end
end

In [22]:
:(get!(TARAI_CACHE, (x, y, z)) do
    $(ex_tarai_naive.args[2])
end)

:(get!(TARAI_CACHE, (x, y, z)) do 
      #= In[22]:2 =#
      begin
          #= In[14]:2 =#
          if x ≤ y
              #= In[14]:3 =#
              y
          else
              #= In[14]:5 =#
              tarai_naive(tarai_naive(x - 1, y, z), tarai_naive(y - 1, z, x), tarai_naive(z - 1, x, y))
          end
      end
  end)

## メモ化（マクロ利用）

### マクロ定義

```julia
macro simplememoize(ex)
    @assert Meta.isexpr(ex, :function)
    fname = ex.args[1].args[1]
    fname_escaped = esc(fname)
    fargs = esc.(ex.args[1].args[2:end])
    fbody = esc(ex.args[2])
    _cache = gensym(Symbol(fname, "_cache"))
    quote
        const $_cache = Dict{NTuple{$(length(fargs))}, Any}()
        function $fname_escaped($(fargs...))
            get!($_cache, ($(Expr(:tuple, fargs...)))) do
                $fbody
            end
        end
    end
end
```

In [23]:
using SimpleMemoizeSample

In [24]:
@macroexpand @simplememoize function tarai(x, y, z)
    if x ≤ y
        y
    else
        tarai(tarai(x - 1, y, z), tarai(y - 1, z, x), tarai(z - 1, x, y))
    end
end

quote
    #= /home/jovyan/src/SimpleMemoizeSample.jl:13 =#
    const var"#32###tarai_cache#253" = SimpleMemoizeSample.Dict{SimpleMemoizeSample.NTuple{3}, SimpleMemoizeSample.Any}()
    #= /home/jovyan/src/SimpleMemoizeSample.jl:14 =#
    function tarai(x, y, z)
        #= /home/jovyan/src/SimpleMemoizeSample.jl:15 =#
        SimpleMemoizeSample.get!(var"#32###tarai_cache#253", (x, y, z)) do 
            #= /home/jovyan/src/SimpleMemoizeSample.jl:16 =#
            begin
                #= In[24]:2 =#
                if x ≤ y
                    #= In[24]:3 =#
                    y
                else
                    #= In[24]:5 =#
                    tarai(tarai(x - 1, y, z), tarai(y - 1, z, x), tarai(z - 1, x, y))
                end
            end
        end
    end
end

In [25]:
@simplememoize function tarai(x, y, z)
    if x ≤ y
        y
    else
        tarai(tarai(x - 1, y, z), tarai(y - 1, z, x), tarai(z - 1, x, y))
    end
end

tarai (generic function with 1 method)

In [26]:
tarai(4, 2, 0)

4

In [27]:
@time tarai(12, 6, 0)

  0.000117 seconds (767 allocations: 42.016 KiB)


12

In [28]:
@time tarai(14, 7, 0)

  0.000044 seconds (143 allocations: 4.469 KiB)


14

In [29]:
@time tarai(16, 8, 0)

  0.000054 seconds (191 allocations: 5.969 KiB)


16

In [30]:
@time tarai(100, 50, 0)

  0.023568 seconds (85.71 k allocations: 3.580 MiB, 50.73% gc time)


100

## ベンチマーク

In [31]:
using BenchmarkTools

In [32]:
@benchmark tarai_naive(12, 6, 0)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     36.567 ms (0.00% GC)
  median time:      42.587 ms (0.00% GC)
  mean time:        43.730 ms (0.00% GC)
  maximum time:     62.868 ms (0.00% GC)
  --------------
  samples:          115
  evals/sample:     1

In [33]:
# @benchmark tarai_cached(12, 6, 0)
@benchmark begin
    empty!($TARAI_CACHE)
    tarai_cached(12, 6, 0)
end

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     22.856 μs (0.00% GC)
  median time:      23.595 μs (0.00% GC)
  mean time:        26.669 μs (0.00% GC)
  maximum time:     228.192 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [34]:
@benchmark begin
    empty!($TARAI_CACHE)
    tarai_cached(100, 50, 0)
end

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     2.153 ms (0.00% GC)
  median time:      2.254 ms (0.00% GC)
  mean time:        2.424 ms (0.00% GC)
  maximum time:     104.091 ms (0.00% GC)
  --------------
  samples:          2056
  evals/sample:     1

In [35]:
cache_var_name = first(name for name in names(Main, all=true) if occursin("#tarai_cache#", string(name)))
cache_var = getfield(@__MODULE__, cache_var_name)
typeof(cache_var)

Dict{Tuple{T,T,T} where T,Any}

In [36]:
@benchmark begin
    empty!($cache_var)
    tarai(12, 6, 0)
end

BenchmarkTools.Trial: 
  memory estimate:  7.03 KiB
  allocs estimate:  225
  --------------
  minimum time:     59.553 μs (0.00% GC)
  median time:      71.857 μs (0.00% GC)
  mean time:        85.042 μs (0.65% GC)
  maximum time:     5.654 ms (98.25% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [37]:
@benchmark begin
    empty!($cache_var)
    tarai(100, 50, 0)
end

BenchmarkTools.Trial: 
  memory estimate:  687.34 KiB
  allocs estimate:  21995
  --------------
  minimum time:     3.082 ms (0.00% GC)
  median time:      3.659 ms (0.00% GC)
  mean time:        4.330 ms (2.06% GC)
  maximum time:     103.942 ms (0.00% GC)
  --------------
  samples:          1150
  evals/sample:     1